#  Neural Machine Translation (NMT) Project
This project demonstrates a simple NMT model using TensorFlow and NumPy to translate English to French.

In [2]:
!pip install tensorflow numpy

##  Step 1: Install Required Libraries
We start by installing necessary libraries.

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

##  Step 2: Import Libraries
Here we import TensorFlow, NumPy, and required Keras modules.

In [4]:
data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("i am fine", "je vais bien"),
    ("thank you", "merci"),
    ("good morning", "bonjour"),
]

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

for input_text, target_text in data:
    target_text = '\t' + target_text + '\n'  # Start and end tokens
    input_texts.append(input_text)
    target_texts.append(target_text)
    input_characters.update(set(input_text))
    target_characters.update(set(target_text))

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)


##  Step 3: Define Dataset
We define our small set of English-French sentence pairs.

In [5]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0


##  Step 4: Data Preprocessing
We tokenize the text and prepare the input and output sequences.

In [6]:
latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")


In [7]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=2, epochs=100)


Epoch 1/100
5/5 [==============================] - 2s 473ms/sample - loss: 1.8831
Epoch 2/100
5/5 [==============================] - 0s 98ms/sample - loss: 1.8246
Epoch 3/100
5/5 [==============================] - 0s 93ms/sample - loss: 2.2816
Epoch 4/100
5/5 [==============================] - 1s 103ms/sample - loss: 1.7413
Epoch 5/100
5/5 [==============================] - 0s 90ms/sample - loss: 1.6854
Epoch 6/100
5/5 [==============================] - 0s 89ms/sample - loss: 1.6365
Epoch 7/100
5/5 [==============================] - 0s 94ms/sample - loss: 1.5774
Epoch 8/100
5/5 [==============================] - 0s 90ms/sample - loss: 1.4959
Epoch 9/100
5/5 [==============================] - 1s 108ms/sample - loss: 1.6846
Epoch 10/100
5/5 [==============================] - 0s 96ms/sample - loss: 1.4221
Epoch 11/100
5/5 [==============================] - 1s 108ms/sample - loss: 1.3705
Epoch 12/100
5/5 [==============================] - 1s 113ms/sample - loss: 1.2625
Epoch 13/100
5/5 [==

In [8]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)  # Define inference decoder model


##  Step 5: Build the Model
We create the encoder-decoder architecture for translation.

In [9]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def encode_input_text(text):
    encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype="float32")
    for t, char in enumerate(text):
        if char in input_token_index:
            encoder_input[0, t, input_token_index[char]] = 1.0
    return encoder_input

def decode_sequence(input_seq):  # Define function to decode input sequence to target
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.0
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]
    return decoded_sentence.strip()


In [10]:
user_input = "hello"
input_seq = encode_input_text(user_input.lower())
print("English:", user_input)
print("French:", decode_sequence(input_seq))


English: hello
French: bonjour


In [15]:
user_input = "How are you"
input_seq = encode_input_text(user_input.lower())
print("English:", user_input)
print("French:", decode_sequence(input_seq))


English: How are you
French: comment ça va


##  Conclusion
This notebook shows a basic implementation of an NMT model using encoder-decoder architecture.

##  Step 8: Translate New Sentences
Function to decode and translate test sentences.

##  Step 6: Train the Model
Now we compile and fit the model with our training data.